In [94]:
from ydc.tools.import_data import import_businesses, import_reviews
from ydc.features.get_features import get_features
import datetime as dt
from random import sample
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation
import numpy as np

import pandas as pd

In [19]:
(features, df, box, combos, cells, n_ind, n_dist) = get_features(status=True, new_cache=True)

Successfully imported reviews with columns ['business_id' 'date' 'stars' 'real_date']


In [20]:
df_rev = import_reviews(new_cache=True)

Successfully imported reviews with columns ['business_id' 'date' 'review_id' 'stars' 'text' 'type' 'user_id' 'votes'
 'real_date']


In [21]:
latest_date = df_rev.groupby('business_id')['real_date'].max()

In [22]:
def one_year(date_series):
    threshold = date_series.max() - dt.timedelta(days=360)
    return (date_series > threshold).sum()

In [23]:
grouped = df_rev.groupby('business_id')['real_date'].agg({'count': one_year})
result = grouped['count'].apply(lambda x: x.nanosecond)  # Pandas is stupid and interpreted the sum as a timestamp

In [24]:
df_res = df.join(result, on='business_id')

In [53]:
def divide_good_and_bad(dataframe, combos, key_feature, percentage):
    """
    Take the given percentage of the dataframe and mark them as "good"
    Done for each category separately, sort by column "key_feature"
    
    """
    # Initialize dataframe sorted by stars and setup new column "good" with False as default
    dataframe = dataframe.sort(columns=key_feature, ascending=False)
    dataframe['good'] = False
    
    for combo in combos:
        # Every (super/sub) category combo
        total = (df['category']==combo).sum()
        
        # Find n (total number times percentage)
        num_good = int(round(total * percentage))
        dataframe['good']
        
        # Get index from best n
        idx = dataframe.loc[df['category']==combo, :].head(num_good).index
        
        # Set those best ones to true
        dataframe.loc[idx, 'good'] = True
    
    # Restore correct order (sorting scrambled it)
    return dataframe.sort_index()

def divide(dataframe, combos, key_feature, n_classes):
    """
    Divide into n classes
    Done for each category separately, sort by column "key_feature"
    0 is best, n_classes-1 worst
    """
    # Initialize dataframe sorted by stars and setup new column "good" with False as default
    dataframe = dataframe.sort(columns=key_feature, ascending=False)
    dataframe['good'] = n_classes-1
    
    for combo in combos:
        # Every (super/sub) category combo
        total = (df['category']==combo).sum()
        # Find number per class, they are all equally big
        n_each = int(round(total / n_classes))
        
        # Get index 
        idx = dataframe.loc[df['category']==combo, :].index.tolist()
        
        for n in range(n_classes-1):
            class_idx = idx[(n * n_each):((n + 1) * n_each)]
            dataframe.loc[class_idx, 'good'] = n
    
    # Restore correct order (sorting scrambled it)
    return dataframe.sort_index()

In [63]:
businesses = divide(df_res, combos, 'count', 4)

In [27]:
test = df_res.groupby("category")['city'].count()
combo = test.argmax()

In [11]:
def fitness_one(features, df_busi, combo, test_portion, test_rounds, c, gamma status=False):
    features = (features - features.mean()) / features.std()

    score = []
    idx = df_busi[df_busi['category'] == combo].index.tolist()

    for _ in range(test_rounds):
        # Take out 15% of data as random sample to test data with
        idx_sample = sample(idx, round(len(idx)*test_portion))
        idx_train = [index for index in idx if index not in idx_sample]

        subcat_clf = svm.SVC(cache_size=2000, C=c, gamma=gamma, kernel='rbf')

        feat_train = features.loc[idx_train, :].values
        quality_train = df_busi.loc[idx_train, 'good'].values  

        feat_sample = features.loc[idx_sample, :].values
        quality_sample = df_busi.loc[idx_sample, 'good'].values  

        subcat_clf.fit(feat_train, quality_train)
        score.append(subcat_clf.score(feat_sample, quality_sample))

    return np.mean(score)

In [14]:
fitness_one(features, businesses, combo, 0.1, 10, 0.1, 0.1)

TypeError: fitness_one() takes from 5 to 6 positional arguments but 7 were given

In [18]:
def fitness(features, df_busi, combos, test_portion, test_rounds, status=False):
    tot_results = []
    for combo in combos:
        try:
            weight = df_busi.loc[df_busi['category']==combo, 'city'].count()
            score = fitness_one(features, df_busi, combo, test_portion, test_rounds, status)
            tot_results.append((score, weight))
        except:
            pass
    
    tot_score = 0
    tot_weight = 0    
    for item in tot_results:
        if item[1]==0:
            continue  # ignore 
        tot_score += item[0] * item[1]
        tot_weight += item[1]
        
    return tot_score/tot_weight
        

In [19]:
fitness(features, businesses, combos, 0.4, 5)

0.58519942046426543

In [13]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
%matplotlib inline

C_range = np.logspace(-3, 3, 5)
gamma_range = np.logspace(-6, -2, 5)

y = target
X = data

param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(y, n_iter=5, test_size=0.2, random_state=42)
grid = GridSearchCV(svm.SVC(), param_grid=param_grid, cv=cv)
grid.fit(X, y)
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

# plot the scores of the grid
# grid_scores_ contains parameter settings and scores
# We extract just the scores
scores = [x[1] for x in grid.grid_scores_]
scores = np.array(scores).reshape(len(C_range), len(gamma_range))

# Draw heatmap of the validation accuracy as a function of gamma and C
#
# The score are encoded as colors with the hot colormap which varies from dark
# red to bright yellow. As the most interesting scores are all located in the
# 0.92 to 0.97 range we use a custom normalizer to set the mid-point to 0.92 so
# as to make it easier to visualize the small variations of score values in the
# interesting range while not brutally collapsing all the low score values to
# the same color.

plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot,
           norm=MidpointNormalize(vmin=0.2, midpoint=0.92))
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar()
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.title('Validation accuracy')
plt.show()

In [112]:
# Machine
clf = svm.SVC(kernel='rbf', C=1, gamma=0.001)
scaler = StandardScaler()

# Data
businesses = divide(df_res, combos, 'count', 2)
idx = (businesses['category'] == combo)
data = features.loc[idx, :].values
target = businesses.loc[idx, 'good'].values

# Go
cv = StratifiedShuffleSplit(target, n_iter=5, test_size=0.2, random_state=42)
data = scaler.fit_transform(data)
scores = cross_validation.cross_val_score(clf, data, target, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.62 (+/- 0.01)


In [118]:
from sklearn.ensemble import RandomForestClassifier
businesses = divide(df_res, combos, 'count', 2)

# Random forest
clf = RandomForestClassifier(n_estimators=10, max_features=1)
scaler = StandardScaler()

# Data
idx = (businesses['category'] == combo)
data = features.loc[idx, :].values
target = businesses.loc[idx, 'good'].values

# Go
cv = StratifiedShuffleSplit(target, n_iter=5, test_size=0.2, random_state=42)
data = scaler.fit_transform(data)
scores = cross_validation.cross_val_score(clf, data, target, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.58 (+/- 0.01)


In [114]:
clf = RandomForestClassifier(n_estimators=10)
clf.fit(data, target)
feat_names = features.columns
feat_weights = clf.feature_importances_
feat_series = pd.Series(feat_weights, index=feat_names)
feat_series.sort(ascending=False)
feat_series[:10]

reviews_mean                    0.058204
reviews_sum                     0.057223
reviews_std                     0.055087
reviews_median                  0.042378
stars_std                       0.042105
stars_sum                       0.041601
weighted review-count           0.041269
stars_mean                      0.039915
reviews_max                     0.037571
neighbourhood_radius_squared    0.036353
dtype: float64